In [1]:
# Installation of any non native supported libraries

# !pip install beautifulsoup4
# !pip install wget

In [2]:
# Import necessary libraries

import pandas as pd
import requests
from bs4 import BeautifulSoup
import wget
import os
import re 
import openpyxl
import pandas as pd
from datetime import datetime
import calendar

In [41]:
# Your local download directory
download_destination = '/Users/dorin/Documents/Jupyter Notebook Files/CCT - Masters/MSc_Data_Analytics_CA2/Data'

## Download Data

In [ ]:
# Your local download directory
download_destination = '/Users/dorin/Documents/Jupyter Notebook Files/CCT - Masters/MSc_Data_Analytics_CA2/Data'

In [ ]:
# Creating Empty DataFrame for storing compiled data
UK_Data_Masterfile = pd.DataFrame()
UK_Data_Masterfile.to_csv(download_destination + '/UK Data/UK_Building_Materials_CostIndex_Compiled.csv')

In [42]:
URL = 'https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/1148775/23-cs4-_Construction_Building_Materials_-_Tables_March_2023.xlsx'
r = requests.get(URL) 
with open(r'/Users/dorin/Documents/Jupyter Notebook Files/CCT - Masters/MSc_Data_Analytics_CA2/Data/UK Data/FIle_new.xlsx', 'wb') as f:
    f.write(r.content)

In [43]:


# Define variable to load the wookbook
wb = openpyxl.load_workbook("/Users/dorin/Documents/Jupyter Notebook Files/CCT - Masters/MSc_Data_Analytics_CA2/Data/file_name.xlsx")
sheets = wb.sheetnames
ws = wb[sheets[3]]
df = pd.DataFrame(ws.values)

df.head(50)

,0,1,2,3,4,5,6,7,8,9,...,18,19,20,21,22,23,24,25,26,27
0,PRICES,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,PRICES
1,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,Table 2: Price Indices of Construction Materials,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,United Kingdom,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,2015=100
5,None,2022,,None,,None,,None,,None,...,None,,None,,None,2023,None,,None,None
6,None,Feb,Mar,None,Apr,None,May,None,Jun,None,...,None,Nov,None,Dec,None,Jan,None,Feb,None,None
7,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
8,AGGREGATES,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,p,None,p,None,AGGREGATES
9,"Gravel, Sand, Clays & Kaolin - exc Aggregate ...",123.3,123.8,None,132.7,None,139.3,None,139.5,None,...,None,142.7,None,142.7,None,142.5,None,141.7,None,"Gravel, Sand, Clays & Kaolin - exc Aggregate ..."


In [44]:
rows_to_delete = [0,1,2,3,4,5,7,8,12,13,19,20,24,25,32,33,41,42,47,48,58]

df = df.drop(index=rows_to_delete)
df.reset_index(drop=True, inplace=True) # Reset the index 

In [45]:
df = df.drop(df.tail(15).index)

In [46]:
df.tail(50)

,0,1,2,3,4,5,6,7,8,9,...,18,19,20,21,22,23,24,25,26,27
0,None,Feb,Mar,None,Apr,None,May,None,Jun,None,...,None,Nov,None,Dec,None,Jan,None,Feb,None,None
1,"Gravel, Sand, Clays & Kaolin - exc Aggregate ...",123.3,123.8,None,132.7,None,139.3,None,139.5,None,...,None,142.7,None,142.7,None,142.5,None,141.7,None,"Gravel, Sand, Clays & Kaolin - exc Aggregate ..."
2,...,129.4,130.4,None,137.3,None,166.2,None,165.3,None,...,None,164,r,164,None,164,None,164,None,...
3,Bituminous Mixtures based on Natural & Artific...,122.2,122.3,None,126.4,None,127.9,None,128.8,None,...,r,139.5,r,139.5,None,140.1,None,139.9,None,Bituminous Mixtures based on Natural & Artific...
4,Cement,119.9,121.7,None,124.3,None,124.3,None,127.2,None,...,None,134.4,None,138.3,None,143.3,None,143.6,None,Cement
5,Ready-mixed concrete **,117.2,119.6,r,120.2,r,126.3,r,128.5,r,...,r,134.3,r,136,None,137,None,137,None,Ready-mixed concrete **
6,Pre-cast concrete products,148,148.5,None,150.5,None,161.6,None,162.2,None,...,r,169.6,r,170.2,None,172.8,None,178.2,None,Pre-cast concrete products
7,"of which : Blocks, bricks, tiles & fl...",138.2,138.5,None,140.6,None,148.3,None,147.7,None,...,None,150.8,r,151.7,None,156.1,None,161.9,None,"of which : Blocks, bricks, tiles & fl..."
8,Concrete reinforcing bars (steel),169.7,213.8,None,240.7,None,242.2,None,239.8,None,...,None,199.9,None,199.6,p,185.2,p,185.8,p,Concrete reinforcing bars (steel)
9,All Bricks,c,c,None,c,None,c,None,c,None,...,None,c,None,c,None,c,None,c,None,All Bricks


In [47]:
df = df.T.set_index(0).T # Assign top row as column headers

In [48]:
df.columns.values[0] = 'Materials' # Label index column

In [49]:
df = df.drop([col for col in df.columns if not col], axis=1)

In [50]:
df.head()

,Materials,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,Jan,Feb
1,"Gravel, Sand, Clays & Kaolin - exc Aggregate ...",123.3,123.8,132.7,139.3,139.5,137.6,137.6,139,136.9,142.7,142.7,142.5,141.7
2,...,129.4,130.4,137.3,166.2,165.3,162.2,161.5,161.5,161.5,164,164,164,164
3,Bituminous Mixtures based on Natural & Artific...,122.2,122.3,126.4,127.9,128.8,135.8,137.7,142.3,141.7,139.5,139.5,140.1,139.9
4,Cement,119.9,121.7,124.3,124.3,127.2,130.7,131.1,132.3,132.6,134.4,138.3,143.3,143.6
5,Ready-mixed concrete **,117.2,119.6,120.2,126.3,128.5,128.8,129.3,131.3,132.1,134.3,136,137,137


In [51]:
dfx = df

In [52]:
from datetime import datetime
current_date = datetime.today()
current_year = int(current_date.strftime('%Y'))
current_month = int(current_date.strftime('%m'))
print(current_month)

5


In [53]:
import calendar
d = dict(enumerate(calendar.month_name))
print(d)

{0: '', 1: 'January', 2: 'February', 3: 'March', 4: 'April', 5: 'May', 6: 'June', 7: 'July', 8: 'August', 9: 'September', 10: 'October', 11: 'November', 12: 'December'}


In [54]:
i = 3
calendar.month_name[i]

'March'

In [55]:
# format dates correctly

col_list = [col for col in dfx.columns[::-1]]
new_col = []

for c in col_list:
    i = i - 1
    if i == 0:
        current_year = current_year - 1
        current_month = 12
        continue
        
    c = str(current_year) + ' ' + calendar.month_name[i]
    print(c)
    new_col.append(c)
    
print(new_col)

2023 February
2023 January
2022 December
2022 November
2022 October
2022 September
2022 August
2022 July
2022 June
2022 May
2022 April
2022 March
2022 February
['2023 February', '2023 January', '2022 December', '2022 November', '2022 October', '2022 September', '2022 August', '2022 July', '2022 June', '2022 May', '2022 April', '2022 March', '2022 February']


In [56]:
new_col.append('Materials') 
new_col.reverse()
print(new_col)

['Materials', '2022 February', '2022 March', '2022 April', '2022 May', '2022 June', '2022 July', '2022 August', '2022 September', '2022 October', '2022 November', '2022 December', '2023 January', '2023 February']


In [57]:
dfx.columns = new_col

In [58]:
dfx

,Materials,2022 February,2022 March,2022 April,2022 May,2022 June,2022 July,2022 August,2022 September,2022 October,2022 November,2022 December,2023 January,2023 February
1,"Gravel, Sand, Clays & Kaolin - exc Aggregate ...",123.3,123.8,132.7,139.3,139.5,137.6,137.6,139,136.9,142.7,142.7,142.5,141.7
2,...,129.4,130.4,137.3,166.2,165.3,162.2,161.5,161.5,161.5,164,164,164,164
3,Bituminous Mixtures based on Natural & Artific...,122.2,122.3,126.4,127.9,128.8,135.8,137.7,142.3,141.7,139.5,139.5,140.1,139.9
4,Cement,119.9,121.7,124.3,124.3,127.2,130.7,131.1,132.3,132.6,134.4,138.3,143.3,143.6
5,Ready-mixed concrete **,117.2,119.6,120.2,126.3,128.5,128.8,129.3,131.3,132.1,134.3,136,137,137
6,Pre-cast concrete products,148,148.5,150.5,161.6,162.2,163.2,163.3,165.3,168.1,169.6,170.2,172.8,178.2
7,"of which : Blocks, bricks, tiles & fl...",138.2,138.5,140.6,148.3,147.7,149,148.9,149.5,151.4,150.8,151.7,156.1,161.9
8,Concrete reinforcing bars (steel),169.7,213.8,240.7,242.2,239.8,222,216.7,216.2,201.6,199.9,199.6,185.2,185.8
9,All Bricks,c,c,c,c,c,c,c,c,c,c,c,c,c
10,Ceramic tiles *,c,c,c,c,c,c,c,c,c,c,c,c,c


In [59]:
mats = dfx['Materials']
dfx = dfx[dfx.columns[:0:-1]]
result = pd.concat([mats, dfx], axis=1)
result

,Materials,2023 February,2023 January,2022 December,2022 November,2022 October,2022 September,2022 August,2022 July,2022 June,2022 May,2022 April,2022 March,2022 February
1,"Gravel, Sand, Clays & Kaolin - exc Aggregate ...",141.7,142.5,142.7,142.7,136.9,139,137.6,137.6,139.5,139.3,132.7,123.8,123.3
2,...,164,164,164,164,161.5,161.5,161.5,162.2,165.3,166.2,137.3,130.4,129.4
3,Bituminous Mixtures based on Natural & Artific...,139.9,140.1,139.5,139.5,141.7,142.3,137.7,135.8,128.8,127.9,126.4,122.3,122.2
4,Cement,143.6,143.3,138.3,134.4,132.6,132.3,131.1,130.7,127.2,124.3,124.3,121.7,119.9
5,Ready-mixed concrete **,137,137,136,134.3,132.1,131.3,129.3,128.8,128.5,126.3,120.2,119.6,117.2
6,Pre-cast concrete products,178.2,172.8,170.2,169.6,168.1,165.3,163.3,163.2,162.2,161.6,150.5,148.5,148
7,"of which : Blocks, bricks, tiles & fl...",161.9,156.1,151.7,150.8,151.4,149.5,148.9,149,147.7,148.3,140.6,138.5,138.2
8,Concrete reinforcing bars (steel),185.8,185.2,199.6,199.9,201.6,216.2,216.7,222,239.8,242.2,240.7,213.8,169.7
9,All Bricks,c,c,c,c,c,c,c,c,c,c,c,c,c
10,Ceramic tiles *,c,c,c,c,c,c,c,c,c,c,c,c,c


In [22]:
URL = 'https://www.data.gov.uk/dataset/75ee36ed-21f7-4d7b-9e7c-f5bf4546145d/monthly-statistics-of-building-materials-and-components'
r = requests.get(URL) 
soup = BeautifulSoup(r.content, "html.parser")
with open(r'/Users/dorin/Documents/Jupyter Notebook Files/CCT - Masters/MSc_Data_Analytics_CA2/Data/example_data.xlsx', 'wb') as f:
    f.write(r.content)

filenames = ['March']   
year = ['2023']
list_of_urls_found = []
for a in soup.select(','.join('a:contains("{}")'.format(i) for i in filenames)):
        list_of_urls_found.append(a['href'])

for l in list_of_urls_found:
    if re.findall('March.*2018'.lower(), l.lower()):
        print(l)

https://www.gov.uk/government/statistics/building-materials-and-components-statistics-march-2018


/Users/dorin/opt/anaconda3/lib/python3.9/site-packages/soupsieve/css_parser.py:876: FutureWarning: The pseudo class ':contains' is deprecated, ':-soup-contains' should be used moving forward.
  warnings.warn(


In [92]:
# Your local download directory
download_destination = '/Users/dorin/Documents/Jupyter Notebook Files/CCT - Masters/MSc_Data_Analytics_CA2/Data'

In [93]:
# Creating Empty DataFrame for storing compiled data
UK_Data_Masterfile = pd.DataFrame()
UK_Data_Masterfile.to_csv(download_destination + '/UK Data/UK_Building_Materials_CostIndex_Compiled.csv')

In [94]:
# Webpage
URL = 'https://www.data.gov.uk/dataset/75ee36ed-21f7-4d7b-9e7c-f5bf4546145d/monthly-statistics-of-building-materials-and-components'
r = requests.get(URL) 
soup = BeautifulSoup(r.content, "html.parser")
list_of_urls_found = []

# Current Date
current_date = datetime.today()
current_year = int(current_date.strftime('%Y'))
current_month = int(current_date.strftime('%m'))
current_month = current_month - 2 # Account for delay uploading data at beginning of month by data.gov.uk
start_year = current_year

# Naming Convension
file_name = ''

def Data_download():
    global file_name
    file_name = '{}/UK Data/{} {} UK Monthly Statistics of Building Materials and Components.xlsx'.format(download_destination, current_year,calendar.month_name[current_month])
    list_of_urls_found = []
    for a in soup.select(','.join('a:contains("{}")'.format(i) for i in calendar.month_name[current_month])):
            list_of_urls_found.append(a['href'])
    for l in list_of_urls_found:
        if re.findall('{}.*{}'.format(calendar.month_name[current_month], current_year).lower(), l.lower()):
            download_link = l
            print(download_link)
    
    # Download link
    r = requests.get(download_link) 
    with open(file_name, 'wb') as f:
        f.write(r.content)
        
    Format_and_Compile()

    
    
def Format_and_Compile():
    # Define variable to load the wookbook
    wb = openpyxl.load_workbook(file_name)
    sheets = wb.sheetnames
    ws = wb[sheets[3]]
    df = pd.DataFrame(ws.values)
    # Delete filler rows
    rows_to_delete = [0,1,2,3,4,5,7,8,12,13,19,20,24,25,32,33,41,42,47,48,58]
    df = df.drop(index=rows_to_delete)
    df.reset_index(drop=True, inplace=True) # Reset the index 
    df = df.drop(df.tail(15).index)
    # Set Column Headers
    df = df.T.set_index(0).T # Assign top row as column headers
    df.columns.values[0] = 'Materials' # Label index column
    df = df.drop([col for col in df.columns if not col], axis=1) # Delete Filler Columns

    # Change Header Format to YYYY-Month
    global current_month
    global current_year
    curr_month = current_month
    curr_year = current_year
    col_list = [col for col in df.columns[::-1]]
    new_col = []
    for c in col_list:
        curr_month = curr_month - 1
        if curr_month == 0:
            curr_year = curr_year - 1
            curr_month = 12
            continue    
        c = str(curr_year) + ' ' + calendar.month_name[curr_month]
        new_col.append(c)    
    new_col.append('Materials') 
    new_col.reverse()
    df.columns = new_col
    
    mats = df['Materials']
    df_cols = df[df.columns[:0:-1]]
    df = pd.concat([mats, df_cols], axis=1)

    Concat_data(df)
    
def Concat_data(dataframe):
    if current_year == start_year:
        # Update Masterfile using concat to append new data
        UK_Data_Masterfile = pd.read_csv(download_destination + '/UK Data/UK_Building_Materials_CostIndex_Compiled.csv', index_col=0)
        Masterfile = pd.concat([UK_Data_Masterfile, dataframe])
        Masterfile.to_csv(download_destination + '/UK Data/UK_Building_Materials_CostIndex_Compiled.csv')
    else:
        # Update Masterfile using concat to append new data
        UK_Data_Masterfile = pd.read_csv(download_destination + '/UK Data/UK_Building_Materials_CostIndex_Compiled.csv', index_col=0)
        Masterfile = pd.concat([UK_Data_Masterfile, dataframe.iloc[:, 1:]], axis=1, join='outer')
        Masterfile.to_csv(download_destination + '/UK Data/UK_Building_Materials_CostIndex_Compiled.csv')

while current_year >= 2022:
    print('{} {}'.format(current_year, calendar.month_name[current_month]))
    Data_download()
    current_year = current_year - 1

2023 March
https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/1148775/23-cs4-_Construction_Building_Materials_-_Tables_March_2023.xlsx
2022 March
https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/1065392/22-cs4_-_Construction_Building_Materials_-_Tables_March_2022.xlsx


In [88]:
df.iloc[:, 1:].head()

,2022 February,2022 March,2022 April,2022 May,2022 June,2022 July,2022 August,2022 September,2022 October,2022 November,2022 December,2023 January,2023 February
1,123.3,123.8,132.7,139.3,139.5,137.6,137.6,139,136.9,142.7,142.7,142.5,141.7
2,129.4,130.4,137.3,166.2,165.3,162.2,161.5,161.5,161.5,164,164,164,164
3,122.2,122.3,126.4,127.9,128.8,135.8,137.7,142.3,141.7,139.5,139.5,140.1,139.9
4,119.9,121.7,124.3,124.3,127.2,130.7,131.1,132.3,132.6,134.4,138.3,143.3,143.6
5,117.2,119.6,120.2,126.3,128.5,128.8,129.3,131.3,132.1,134.3,136,137,137


In [63]:
current_date = datetime.today()
current_year = int(current_date.strftime('%Y'))
current_year

2023

In [ ]:
# Define variable to load the wookbook
wb = openpyxl.load_workbook(file_name)
sheets = wb.sheetnames
ws = wb[sheets[3]]
df = pd.DataFrame(ws.values)
# Delete filler rows
rows_to_delete = [0,1,2,3,4,5,7,8,12,13,19,20,24,25,32,33,41,42,47,48,58]
df = df.drop(index=rows_to_delete)
df.reset_index(drop=True, inplace=True) # Reset the index 
df = df.drop(df.tail(15).index)
# Set Column Headers
df = df.T.set_index(0).T # Assign top row as column headers
df.columns.values[0] = 'Materials' # Label index column
df = df.drop([col for col in df.columns if not col], axis=1) # Delete Filler Columns

# Change Header Format to YYYY-Month
col_list = [col for col in df.columns[::-1]]
new_col = []
for c in col_list:
    current_month = current_month - 1
    if current_month == 0:
        current_year = current_year - 1
        current_month = 12
        continue    
    c = str(current_year) + ' ' + calendar.month_name[current_month]
    new_col.append(c)    
new_col.append('Materials') 
new_col.reverse()
df.columns = new_col
df.head()

In [72]:
x = df
y = df

In [65]:
xy = pd.concat([x, y], axis=1, join='outer')
xy

,Materials,2022 February,2022 March,2022 April,2022 May,2022 June,2022 July,2022 August,2022 September,2022 October,...,2022 May,2022 June,2022 July,2022 August,2022 September,2022 October,2022 November,2022 December,2023 January,2023 February
1,"Gravel, Sand, Clays & Kaolin - exc Aggregate ...",123.3,123.8,132.7,139.3,139.5,137.6,137.6,139,136.9,...,139.3,139.5,137.6,137.6,139,136.9,142.7,142.7,142.5,141.7
2,...,129.4,130.4,137.3,166.2,165.3,162.2,161.5,161.5,161.5,...,166.2,165.3,162.2,161.5,161.5,161.5,164,164,164,164
3,Bituminous Mixtures based on Natural & Artific...,122.2,122.3,126.4,127.9,128.8,135.8,137.7,142.3,141.7,...,127.9,128.8,135.8,137.7,142.3,141.7,139.5,139.5,140.1,139.9
4,Cement,119.9,121.7,124.3,124.3,127.2,130.7,131.1,132.3,132.6,...,124.3,127.2,130.7,131.1,132.3,132.6,134.4,138.3,143.3,143.6
5,Ready-mixed concrete **,117.2,119.6,120.2,126.3,128.5,128.8,129.3,131.3,132.1,...,126.3,128.5,128.8,129.3,131.3,132.1,134.3,136,137,137
6,Pre-cast concrete products,148,148.5,150.5,161.6,162.2,163.2,163.3,165.3,168.1,...,161.6,162.2,163.2,163.3,165.3,168.1,169.6,170.2,172.8,178.2
7,"of which : Blocks, bricks, tiles & fl...",138.2,138.5,140.6,148.3,147.7,149,148.9,149.5,151.4,...,148.3,147.7,149,148.9,149.5,151.4,150.8,151.7,156.1,161.9
8,Concrete reinforcing bars (steel),169.7,213.8,240.7,242.2,239.8,222,216.7,216.2,201.6,...,242.2,239.8,222,216.7,216.2,201.6,199.9,199.6,185.2,185.8
9,All Bricks,c,c,c,c,c,c,c,c,c,...,c,c,c,c,c,c,c,c,c,c
10,Ceramic tiles *,c,c,c,c,c,c,c,c,c,...,c,c,c,c,c,c,c,c,c,c


In [66]:
# Update Masterfile using concat to append new data
UK_Data_Masterfile = pd.read_csv(download_destination + '/UK Data/UK_Building_Materials_CostIndex_Compiled.csv', index_col=0)
Masterfile = pd.concat([UK_Data_Masterfile, x], axis=1, join='outer')
Masterfile.to_csv(download_destination + '/UK Data/UK_Building_Materials_CostIndex_Compiled.csv')

In [73]:
# Update Masterfile using concat to append new data
UK_Data_Masterfile = pd.read_csv(download_destination + '/UK Data/UK_Building_Materials_CostIndex_Compiled.csv', index_col=0)
Masterfile = pd.concat([UK_Data_Masterfile, y.iloc[:, 1:]], axis=1, join='outer')
Masterfile.to_csv(download_destination + '/UK Data/UK_Building_Materials_CostIndex_Compiled.csv')